In [1]:
import jieba
import numpy
import jieba.analyse

import pandas as pd

In [2]:
import gensim

from gensim import corpora, models, similarities

In [3]:
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB

## 1. 数据预处理

In [4]:
df_news = pd.read_table('./data/val.txt',
                        names=['category', 'theme', 'URL','content'],
                        encoding='utf-8')

df_news = df_news.dropna()
df_news.head(2)

,category,theme,URL,content
0,汽车,新辉腾 ４．２ Ｖ８ ４座加长Ｉｎｄｉｖｉｄｕａｌ版２０１１款 最新报价,http://auto.data.people.com.cn/model_15782/,经销商 电话 试驾／订车Ｕ憬杭州滨江区江陵路１７８０号４００８－１１２２３３转５８６４＃保常...
1,汽车,９１８ Ｓｐｙｄｅｒ概念车,http://auto.data.people.com.cn/prdview_165423....,呼叫热线 ４００８－１００－３００ 服务邮箱 ｋｆ＠ｐｅｏｐｌｅｄａｉｌｙ．ｃｏｍ．ｃｎ


In [5]:
df_news.shape

(5000, 4)

### 分词操作

In [6]:
content = df_news.content.values.tolist()
content[1000]

'阿里巴巴集团昨日宣布，将在集团管理层面设立首席数据官岗位（Ｃｈｉｅｆ\u3000Ｄａｔａ\u3000Ｏｆｆｉｃｅｒ），阿里巴巴Ｂ２Ｂ公司ＣＥＯ陆兆禧将会出任上述职务，向集团ＣＥＯ马云直接汇报。＞菹ぃ和６月初的首席风险官职务任命相同，首席数据官亦为阿里巴巴集团在完成与雅虎股权谈判，推进“ｏｎｅ\u3000ｃｏｍｐａｎｙ”目标后，在集团决策层面新增的管理岗位。０⒗锛团昨日表示，“变成一家真正意义上的数据公司”已是战略共识。记者刘夏'

In [7]:
content_S = []

for line in content:
    current_segment = jieba.lcut(line)
    
    if len(current_segment) > 1 and current_segment != '\r\n':
        content_S.append(current_segment)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\18917\AppData\Local\Temp\jieba.cache
Loading model cost 0.967 seconds.
Prefix dict has been built succesfully.


In [8]:
df_content = pd.DataFrame({'content_S': content_S})
df_content.head(3)

,content_S
0,"[经销商, , 电话, , 试驾, ／, 订车, Ｕ, 憬, 杭州, 滨江区, 江陵, ..."
1,"[呼叫, 热线, , ４, ０, ０, ８, －, １, ０, ０, －, ３, ０, ０..."
2,"[Ｍ, Ｉ, Ｎ, Ｉ, 品牌, 在, 二月, 曾经, 公布, 了, 最新, 的, Ｍ, Ｉ..."


### 去停用词

In [9]:
def drop_stopwords(contents, stopwords):
    contents_clean = []
    all_words = []
    
    for line in contents:
        line_clean = []
        
        for word in line:
            if word in stopwords:
                continue
    
            line_clean.append(word)
            all_words.append(str(word))

        contents_clean.append(line_clean)

    return contents_clean,all_words

In [10]:
stopwords = pd.read_csv("./data/stopwords.txt",
                        index_col=False,
                        sep="\t",
                        quoting=3,
                        names=['stopword'],
                        encoding='utf-8')
stopwords.head(3)

,stopword
0,!
1,""""
2,#


In [11]:
contents = df_content['content_S'].values.tolist()    
stopwords = stopwords['stopword'].values.tolist()

contents_clean, all_words = drop_stopwords(contents, stopwords)

In [12]:
df_all_words = pd.DataFrame({'all_words': all_words})
df_all_words.head(3)

,all_words
0,经销商
1,电话
2,试驾


### TF-IDF：提取关键词

In [13]:
index = 1000
print(df_news['content'][index])

content_S_str = "".join(content_S[index])  
print ("  ".join(jieba.analyse.extract_tags(content_S_str, topK=10, withWeight=False)))

阿里巴巴集团昨日宣布，将在集团管理层面设立首席数据官岗位（Ｃｈｉｅｆ　Ｄａｔａ　Ｏｆｆｉｃｅｒ），阿里巴巴Ｂ２Ｂ公司ＣＥＯ陆兆禧将会出任上述职务，向集团ＣＥＯ马云直接汇报。＞菹ぃ和６月初的首席风险官职务任命相同，首席数据官亦为阿里巴巴集团在完成与雅虎股权谈判，推进“ｏｎｅ　ｃｏｍｐａｎｙ”目标后，在集团决策层面新增的管理岗位。０⒗锛团昨日表示，“变成一家真正意义上的数据公司”已是战略共识。记者刘夏
阿里巴巴  集团  首席  岗位  数据  层面  职务  陆兆禧  官亦为  刘夏


## 2. LDA主题模型

In [14]:
# 做映射，相当于词袋

dictionary = corpora.Dictionary(contents_clean)
corpus = [dictionary.doc2bow(sentence) for sentence in contents_clean]

In [15]:
# 类似Kmeans自己指定K值

lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

In [16]:
# 一号分类结果

print(lda.print_topic(1, topn=5))

0.013*"中" + 0.010*"卫视" + 0.006*"考生" + 0.005*"饰演" + 0.004*"网友"


In [17]:
for topic in lda.print_topics(num_topics=20, num_words=5):
    print (topic[1])

0.010*"中国" + 0.006*"节目" + 0.006*"中" + 0.006*"电影" + 0.005*"观众"
0.013*"中" + 0.010*"卫视" + 0.006*"考生" + 0.005*"饰演" + 0.004*"网友"
0.006*"中" + 0.004*"球队" + 0.004*"Ｃ" + 0.003*"赛季" + 0.003*"雅虎"
0.005*"中" + 0.005*"比赛" + 0.004*"孩子" + 0.004*"说" + 0.004*"祝福"
0.009*"饰演" + 0.007*"万" + 0.006*"中" + 0.006*"号" + 0.004*"说"
0.017*"该剧" + 0.008*"爆料" + 0.007*"男人" + 0.006*"大戏" + 0.004*"青瓷"
0.011*"皮肤" + 0.009*"肌肤" + 0.007*"男人" + 0.006*"乳房" + 0.005*"女人"
0.007*"导演" + 0.005*"中" + 0.005*"电影" + 0.005*"说" + 0.004*"故事"
0.004*"中" + 0.004*"飞行" + 0.003*"说" + 0.003*"发展" + 0.003*"Ｎ"
0.005*"中" + 0.004*"食品" + 0.003*"万" + 0.003*"说" + 0.003*"工作"
0.009*"中国" + 0.007*"中" + 0.006*"张绍" + 0.005*"美国" + 0.003*"Ｐ"
0.004*"说" + 0.004*"李小璐" + 0.004*"剧组" + 0.004*"时尚" + 0.003*"中"
0.007*"中国" + 0.005*"离婚" + 0.004*"中" + 0.004*"男人" + 0.004*"导演"
0.006*"中" + 0.005*"中国" + 0.005*"爆料" + 0.004*"说" + 0.002*"Ｉ"
0.018*"ａ" + 0.018*"ｅ" + 0.013*"ｉ" + 0.013*"ｏ" + 0.012*"ｎ"
0.005*"赛区" + 0.005*"奢侈品" + 0.004*"纹身" + 0.004*"排毒" + 0.004*"肌肤"
0.012*"撒" + 0.010*"观众

## 3. 贝叶斯分类

In [18]:
df_train = pd.DataFrame({'contents_clean': contents_clean, 'label': df_news['category']})
df_train.head(3)

,contents_clean,label
0,"[经销商, 电话, 试驾, 订车, Ｕ, 憬, 杭州, 滨江区, 江陵, 路, 号, 转, ...",汽车
1,"[呼叫, 热线, 服务, 邮箱, ｋ, ｆ, ｐ, ｅ, ｏ, ｐ, ｌ, ｅ, ｄ, ａ,...",汽车
2,"[Ｍ, Ｉ, Ｎ, Ｉ, 品牌, 二月, 公布, 最新, Ｍ, Ｉ, Ｎ, Ｉ, 新, 概念...",汽车


### 训练数据

In [19]:
df_train.label.unique()

array(['汽车', '财经', '科技', '健康', '体育', '教育', '文化', '军事', '娱乐', '时尚'],
      dtype=object)

In [20]:
label_mapping = {"汽车": 1, "财经": 2, "科技": 3, "健康": 4, "体育": 5, "教育": 6,"文化": 7,"军事": 8,"娱乐": 9,"时尚": 0}
df_train['label'] = df_train['label'].map(label_mapping)
df_train.head(3)

,contents_clean,label
0,"[经销商, 电话, 试驾, 订车, Ｕ, 憬, 杭州, 滨江区, 江陵, 路, 号, 转, ...",1
1,"[呼叫, 热线, 服务, 邮箱, ｋ, ｆ, ｐ, ｅ, ｏ, ｐ, ｌ, ｅ, ｄ, ａ,...",1
2,"[Ｍ, Ｉ, Ｎ, Ｉ, 品牌, 二月, 公布, 最新, Ｍ, Ｉ, Ｎ, Ｉ, 新, 概念...",1


In [21]:
x_train, x_test, y_train, y_test = train_test_split(df_train['contents_clean'].values, df_train['label'].values, random_state=1)

print(x_train[0][1])

上海


In [22]:
words = []

for line_index in range(len(x_train)):
    try:
        words.append(' '.join(x_train[line_index]))
    except:
        print(line_index, word_index)

print(words[0])
print("Length of words: ", len(words))

中新网 上海 日电 于俊 父亲节 网络 吃 一顿 电影 快餐 微 电影 爸 对不起 我爱你 定于 本月 父亲节 当天 各大 视频 网站 首映 葜 谱 鞣 剑 保慈 障蚣 钦 呓 樯 埽 ⒌ 缬 埃 ǎ 停 椋 悖 颍 铩 妫 椋 恚 称 微型 电影 新 媒体 平台 播放 状态 短时 休闲 状态 观看 完整 策划 系统 制作 体系 支持 显示 较完整 故事情节 电影 微 超短 放映 微 周期 制作 天 数周 微 规模 投资 人民币 几千 数万元 每部 内容 融合 幽默 搞怪 时尚 潮流 人文 言情 公益 教育 商业 定制 主题 单独 成篇 系列 成剧 唇 开播 微 电影 爸 对不起 我爱你 讲述 一对 父子 观念 缺少 沟通 导致 关系 父亲 传统 固执 钟情 传统 生活 方式 儿子 新派 音乐 达 习惯 晚出 早 生活 性格 张扬 叛逆 两种 截然不同 生活 方式 理念 差异 一场 父子 间 拉开序幕 子 失手 打破 父亲 心爱 物品 父亲 赶出 家门 剧情 演绎 父亲节 妹妹 哥哥 化解 父亲 这场 矛盾 映逋坏 嚼 斫 狻 ⒍ 粤 ⒌ 桨容 争执 退让 传统 尴尬 父子 尴尬 情 男人 表达 心中 那份 感恩 一杯 滤挂 咖啡 父亲节 变得 温馨 镁 缬 缮 虾 Ｎ 逄 煳 幕 传播 迪欧 咖啡 联合 出品 出品人 希望 观摩 扪心自问 父亲节 父亲 记得 父亲 生日 哪一天 父亲 爱喝 跨出 家门 那一刻 感觉 一颗 颤动 心 操劳 天下 儿女 父亲节 大声 喊出 父亲 家人 爱 完
Length of words:  3750


### 测试数据

In [23]:
test_words = []

for line_index in range(len(x_test)):
    try:
        test_words.append(' '.join(x_test[line_index]))
    except:
         print (line_index,word_index)

test_words[0]
print("Length of test words: ", len(test_words))

Length of test words:  1250


### 构建词频向量小例子

In [24]:
texts=["dog cat fish", "dog cat cat", "fish bird", "bird"]

cv = CountVectorizer()
cv_fit=cv.fit_transform(texts)

print( cv.get_feature_names())
print( cv_fit.toarray())
print(cv_fit.toarray().sum(axis=0))

['bird', 'cat', 'dog', 'fish']
[[0 1 1 1]
 [0 2 1 0]
 [1 0 0 1]
 [1 0 0 0]]
[2 3 2 2]


### 基于词频向量构建分类器

In [25]:
vec = CountVectorizer(analyzer='word', max_features=4000,  lowercase=False)
vec.fit(words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=4000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [26]:
classifier = MultinomialNB()
classifier.fit(vec.transform(words), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
classifier.score(vec.transform(test_words), y_test)

0.804

### 基于TF-IDF构建分类器

In [28]:
vectorizer = TfidfVectorizer(analyzer='word', max_features=4000,  lowercase=False)
vectorizer.fit(words)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=4000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [29]:
classifier = MultinomialNB()
classifier.fit(vectorizer.transform(words), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
classifier.score(vectorizer.transform(test_words), y_test)

0.8152